Import Python Libraries


In [93]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import numpy as np

Read in Wisconsin Breast Cancer Dataset

In [94]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data", 
                 header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [95]:
df.columns = ["ID", "Diagnosis", "mean_radius", "mean_texture", 
                "mean_perimeter", "mean_area", "mean_smoothness", 
                "mean_compactness", "mean_concavity", "mean_concave_points", 
                "mean_symmetry", "mean_fractal_dimension", "sd_radius", 
                "sd_texture", "sd_perimeter", "sd_area", "sd_smoothness",
                "sd_compactness", "sd_concavity", "sd_concave_points", 
                "sd_symmetry", "sd_fractal_dimension", "worst_radius", 
                "worst_texture", "worst_perimeter", "worst_area", 
                "worst_smoothness", "worst_compactness", "worst_concavity", 
                "worst_concave_points", "worst_symmetry", "worst_fractal_dimension"]
df.head()

,ID,Diagnosis,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,...,worst_radius,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


Split Dataset Into Data Features and Target

In [96]:
df_Target = df["Diagnosis"]
df_Features = df[["mean_radius", "mean_texture", "mean_perimeter", 
                  "mean_area", "mean_smoothness", "mean_compactness", 
                  "mean_concavity", "mean_concave_points", "mean_symmetry", 
                  "mean_fractal_dimension", "sd_radius", "sd_texture", 
                  "sd_perimeter", "sd_area", "sd_smoothness", "sd_compactness", 
                  "sd_concavity", "sd_concave_points", "sd_symmetry", 
                  "sd_fractal_dimension", "worst_radius", "worst_texture", 
                  "worst_perimeter", "worst_area", "worst_smoothness", 
                  "worst_compactness", "worst_concavity", "worst_concave_points", 
                  "worst_symmetry", "worst_fractal_dimension"]]

Set kNN's k value to 3 and fit data

In [113]:
knn = KNeighborsClassifier(n_neighbors=3,
                           weights="uniform",
                           p=2,
                           metric="minkowski")

Y = df_Target.values
X = df_Features.values

from sklearn import cross_validation
cv_indices = cross_validation.StratifiedKFold(Y, n_folds=3)

def accuracy_crossvalidator(X, Y, knn, cv_indices):
    scores = []
    for train_i, test_i in cv_indices:
        X_train = X[train_i, :]
        X_test = X[test_i, :]

        Y_train = Y[train_i]
        Y_test = Y[test_i]

        knn.fit(X_train, Y_train)
        
        acc = knn.score(X_test, Y_test)
        scores.append(acc)        
        print('Fold accuracy:', acc)
    print('Mean CV accuracy:', np.mean(scores))

Find Model Accuracy (score)

In [114]:
accuracy_crossvalidator(X, Y, knn, cv_indices)

('Fold accuracy:', 0.89473684210526316)
('Fold accuracy:', 0.95263157894736838)
('Fold accuracy:', 0.91534391534391535)
('Mean CV accuracy:', 0.92090411213218226)


Prepare Test Data and Predict

In [115]:
X1 = df_Features
Y1 = df_Target

knn.fit(X,Y)
knn.predict(X1)


array(['M', 'M', 'M', 'B', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'B', 'M', 'M', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'B', 'B',
       'B', 'M', 'M', 'M', 'M', 'M', 'M', 'B', 'M', 'B', 'B', 'B', 'B',
       'B', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'M',
       'M', 'B', 'B', 'B', 'B', 'M', 'B', 'M', 'M', 'B', 'M', 'B', 'M',
       'M', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'M', 'B', 'B', 'B',
       'B', 'M', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'B',
       'B', 'M', 'B', 'B', 'M', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'M', 'M', 'M', 'B', 'M', 'M', 'B', 'B', 'B', 'M', 'M', 'B', 'M',
       'B', 'M', 'M', 'B', 'M', 'B', 'B', 'B', 'M', 'B', 'B', 'M', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'M', 'B', 'B', 'B', 'B', 'M', 'M', 'B', 'M', 'B', 'B', 'M', 'M',
       'B', 'B', 'M', 'M', 'B', 'B', 'B', 'B', 'M', 'B', 'B', 'M

Build Confusion Matrix

In [116]:
y_pred = knn.predict(X1)
y_true = df_Target.values

In [1]:
confusion_matrix(y_true, y_pred)

NameError: name 'confusion_matrix' is not defined